# In this Notebook is presented typical use-cases for Vault recipient
#### this example demonstrates *confidential-storage* implementation protocol https://identity.foundation/confidential-storage/#ecosystem-overview
1. Configure **Wallet** and **Secrets**
2. Establish P2P connection with party who plays **Confidential Storage** role

<span style='color:blue; font-size: 120%;'>**&darr; below is placed button** to generate invitation link. Copy and Paste **this link** to Vault actor side</span>


In [63]:
import asyncio
import threading
from typing import Optional

import iplantuml
import sirius_sdk
import ipywidgets as widgets


################### P2P connection to Confidential-Storage Provider ###################
PROVIDER: Optional[sirius_sdk.Pairwise] = None
#######################################################
################### Shared Secrets (encrypt/dectypt with them) ############################
pk, sk = sirius_sdk.encryption.create_keypair(seed=b'0000000000000000000000000000DEMO')
PK_SHARED = sirius_sdk.encryption.bytes_to_b58(pk)
SK_SHARED = sirius_sdk.encryption.bytes_to_b58(sk)
###############################################################

# Init connection to Wallet that maintained in the Cloud
# Off cource developer have alternative - he may maintain Wallet and secrets on other way (KERI library) 
# thanks to Sirius-SDK is friendly to Legacy and flexible due to dependency injection architecture 

CFG = sirius_sdk.Config().setup_cloud(
    server_uri='https://agents.socialsirius.com',
    credentials='s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rUj54zpQ9fWOm3lsdQwI3d2EzIARf+kcCNKU5wKJX9LY=',
    p2p={
        "their_verkey": "FBHVE36ZfixjBwqJWKy45sTRJdt9KDFrqQPL6rExJweN",
        "my_keys": [
          "22UJ1f3nPuJR9y6znPXwGw2xEyzmNRigwbfNoJPraPtY",
          "5JjNff6y8GxWtWxWA5BiKkfZVx4hYQHscwZt7z613CrENWL7xRMehy5MWNy6BqooX73PmDDSso9npNEqsmbXgXCG"
        ]
   }
)


#################################################################################
######################## Connect to Vaults Provider #############################
#################################################################################
async def establish_connection():
    global PROVIDER
    try:
        async with sirius_sdk.context(CFG):
            did, verkey = await sirius_sdk.DID.create_and_store_my_did()
            manager = sirius_sdk.recipes.InvitationManager(
                me=sirius_sdk.Pairwise.Me(did, verkey),
                my_label='Vault Client'
            )
            invitation = await manager.make_invitation()
            invitation_url = 'https://example.com/invitation' + invitation.invitation_url
            output.append_stdout('\n !!!!!! Copy invitation and PASS to Provider script !!!!!!\n')
            output.append_stdout('_'*64 + '\n')
            output.append_stdout(invitation_url)
            output.append_stdout('\n' + '_'*64)
            
            # Start ti listen connection requests
            output.append_stdout('\nWait for connection requests ...\n')
            PROVIDER = await manager.wait_connection(invitation)
            output.append_stdout('_'*64 + '\n')
            output.clear_output()
            output.append_stdout(f'\n!!! Connection with "{PROVIDER.their.label}" was established !!!!')
            output.append_stdout('\n' + '_'*64)
    except Exception as e:
        output.append_stderr(repr(e))
#################################################################################
#################################################################################
#################################################################################


output = widgets.Output(layout={"width": "600px"})
button = widgets.Button(description="Generate Invitation", button_style='primary')
button_clipboard = widgets.Button(description="Copy to clipboard")

def on_button_clicked(b):
    output.clear_output()
    asyncio.ensure_future(establish_connection())
    
button.on_click(on_button_clicked)
        
display(button, output)


Button(button_style='primary', description='Generate Invitation', style=ButtonStyle())

Output(layout=Layout(width='600px'))

<span style='color:blue; font-size: 120%;'>**&uarr; above is placed button** to generate invitation link. Copy and Paste **this link** to Vault actor side</span>

 - Navigate to Demo script https://github.com/Sirius-social/sirius-sdk-python/tree/master/how-tos/confidential_storage and run by typing ```python3 manage.py```
 - After that wait to establish **P2P** connection among ours and **Confidential StorageVault**

## Code below demonstrate how to request all supported Vaults
We provide in this **Demo** two types of 
 1. **Encrypted** vault: all data is encrypted with Storage internal keys
 2. **NonEncrypted** vault: all data is stored locally on storage side as they was recieved without any postprocessing
 

In [62]:
import json

from IPython.display import JSON, HTML


assert PROVIDER is not None, "P2P is empty, establish connection with Storage provider"

VAULTS = []
VAULT_ID: Optional[str] = None


#################################################################################
###################### UI Controls ##############################################
#################################################################################
# Vaults selection widgets
vaults_dropdown = widgets.Dropdown(description='Vault ID:')
vaults_dropdown.disabled = True
# Log outputs
output_log = widgets.Output(layout={"width": "800px", "border": "1px solid #dddddd", "padding": "10px"})
output_vaults = widgets.Output(layout={"height": "70px", "border": "1px solid #dddddd", "padding": "10px"})
button_list = widgets.Button(description="Query Vaults", button_style='primary')
vboxes = []
cols = [
    # Each tuple contains a column header and a list of items/widgets
    (
        'Operations', 
        [
            widgets.HBox([vaults_dropdown, button_list]),
            output_vaults
        ]
        
    ),
    (
        'Log', 
        [output_log]
    ),
]
for header, data in cols:
    vboxes.append(widgets.VBox([widgets.HTML('<b>%s</b>' % header)] + [
        d if isinstance(d, widgets.Widget) else widgets.HTML(str(d)) for d in data],
    layout=widgets.Layout(border='1px solid #DDDDDD')))
hbox = widgets.HBox(vboxes)
#################################################################################
#################################################################################
#################################################################################


#################################################################################
######################### Retrieve Vault list ###################################
#################################################################################
async def query_vaults():
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            vaults = await protocol.list_vaults()
            for cfg in vaults:
                js = cfg.as_json()
                VAULTS.append(js)
                output_log.append_stdout(f'\n=========== ID: {js["id"]}=============')
                output_log.append_stdout('\n' + json.dumps(js, indent=2))
            vaults_dropdown.options = [vault['id'] for vault in VAULTS]
            vaults_dropdown.value = VAULTS[0]['id']
            vaults_dropdown.disabled = False
            button_list.disabled = True
    except Exception as e:
        output_log.append_stderr(repr(e))
#################################################################################
#################################################################################
#################################################################################
        
        
#################################################################################
########################## Select Vault #########################################
#################################################################################
async def open_vault(vault_id: str, vault_id_to_close: str = None):
    global VAULT_ID
    await asyncio.sleep(0.1)
    try:
        async with sirius_sdk.context(CFG):
            if vault_id_to_close:
                # Close old session
                output_vaults.append_display_data(HTML(f'<span style="color:red;">UnSelect: [{vault_id_to_close}]</span>'))
            # Select Vault
            VAULT_ID = vault_id
            output_vaults.append_display_data(HTML(f'<span style="color:green;">Selected Vault: [{vault_id}]</span>'))
    except Exception as e:
        output_vaults.append_stderr(repr(e))
#################################################################################
#################################################################################
#################################################################################


def clear_ui():
    output_log.clear_output()
    output_vaults.clear_output()
    vaults_dropdown.options = []
    

def on_btn_list_clicked(b):
    VAULTS.clear()
    clear_ui()
    asyncio.ensure_future(query_vaults())
    
    
def on_vaults_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        output_vaults.clear_output()
        asyncio.ensure_future(open_vault(vault_id=change['new'], vault_id_to_close=change['old']))
    
    
button_list.on_click(on_btn_list_clicked)
vaults_dropdown.observe(on_vaults_dropdown_change)
display(hbox)


AssertionError: P2P is empty, establish connection with Storage provider

<span style='color:blue; font-size: 120%;'>**&uarr;** select corresponding **vault** you want to operate with</span>
>*Notice*: If storage doc/stream is **shared** (for example it has public **URL** link) then you can **share** this doc **only** if vault provider **does not apply encryption** on self-side

<img src="https://identity.foundation/confidential-storage/diagrams/SDS_Layers.svg" style="max-height:400px;"/>

Let's consider **THREE cases** of vault enctyption approach with usefull outcomes 
 - **Case-1**: Vault provider maintain files on self side and don't share it with public URL links. In this case encryption on vault side is additional mechanism of protection. According to **GDPR** roles, provider acts as **Data SubProcessor**. Access to docs & streams restricted with **P2P** relationship between *Client* and *Vault Provider*
 - **Case-2**: Vault provider **encrypts** files on self side but **share** them with public URL links. In this case *Client* may share doc/stream with any participant with **one precondition**: participnt **should have secrets** to decrypt data.
 - **Case-3**: Vault provider **don't encrypt** files on self side and **share** them with public URL links. IN this case *Client* may share data with others **avoiding any preconditions**. Be careful, bacause *Vault Provider* don't have responsibility to protect data, opposit it became responsibility *Client* and it *Participants* 

## Create/Load/Remove documents
Code below demonstrate scenarios for:
 1. **Creation of documents** with encryption option on *Client* side. Pay attention for files semantic on *Vault Provider* side for *encrypted* vault configuration and *non-encrypted* ones.
 2. **Document loading** - pay attention for JSON structures for *encrypted* and *non-encrypted* doc configurations
 3. **Removing** docs

In [61]:
assert VAULT_ID is not None, "You should to select Vault!"


#################################################################################
####################### UI CONTROLS #############################################
#################################################################################
button_create_doc = widgets.Button(description="> Create Document", button_style='success')
button_remove_doc = widgets.Button(description="> Remove Document", button_style='danger')
button_list_docs = widgets.Button(description="List Documents >", button_style='primary')
checkbox_encrypt_doc = widgets.Checkbox(False, description='Encrypt')
dropdown_enc_type_doc = widgets.Dropdown(
    options=['Wallet', 'SharedKeys'],
    value='Wallet',
    description='EncType:',
    disabled=False,
)
dropdown_enc_type_doc.layout.display = 'none'
input_create_doc = widgets.Text(
    value='my_document.txt',
    placeholder='Type document name',
    description='Name:  ',
)
input_doc_content = widgets.Textarea(
    value='This is document content',
    placeholder='Fill document content',
    description='Content'
)
input_remove_doc = widgets.Text(
    placeholder='Put here document id or urn',
    description='ID or URN: '
)
output_log_docs = widgets.Output(layout={"width": "800px", "border": "1px solid #eeeeee", "padding": "10px"})
vboxes = []

grid_layout = widgets.GridspecLayout(6, 2)
# fill it in with widgets
grid_layout[0, 0] = input_create_doc
grid_layout[0, 1] = button_create_doc
grid_layout[1, 0] = input_doc_content
grid_layout[2, 0] = checkbox_encrypt_doc
grid_layout[3, 0] = dropdown_enc_type_doc
grid_layout[4, 1] = button_list_docs
grid_layout[5, 0] = input_remove_doc
grid_layout[5:, 1] = button_remove_doc
#grid_layout.layout.width = '500px'
#################################################################################
#################################################################################
#################################################################################


cols = [
    # Each tuple contains a column header and a list of items/widgets
    (
        'Doc Operations', 
        [grid_layout]
        
    ),
    (
        'Doc Logs', 
        [output_log_docs]
    ),
]
for header, data in cols:
    vboxes.append(widgets.VBox([widgets.HTML('<b>%s</b>' % header)] + [
        d if isinstance(d, widgets.Widget) else widgets.HTML(str(d)) for d in data],
    layout=widgets.Layout(border='1px solid #DDDDDD')))
hbox = widgets.HBox(vboxes)


#################################################################################
################### Document creation, saving, loading ##########################
#################################################################################
async def create_document(name: str, content: bytes):
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # Create document
            structored_doc = await protocol.create_document(uri=name, category='document')
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=structored_doc, sequence=0).as_json()
            output_log_docs.append_display_data(HTML("<b style='color: blue;'>Create: Structured document</b>"))
            output_log_docs.append_stdout(json.dumps(jsonable, indent=2))
            
            # Save document
            if checkbox_encrypt_doc.value:
                if dropdown_enc_type_doc.value == 'Wallet':
                    output_log_docs.append_stdout('\n++++++++++++++++ Encrypt via Wallet +++++++++++++++')
                    encrypted_doc = sirius_sdk.aries_rfc.EncryptedDocument(target_verkeys=[PROVIDER.me.verkey])
                else:
                    output_log_docs.append_stdout('\n++++++++++++++++ Encrypt via Shared Keys +++++++++++++')
                    encrypted_doc = sirius_sdk.aries_rfc.EncryptedDocument(target_verkeys=[PK_SHARED])
                encrypted_doc.content = content
                await encrypted_doc.encrypt()
            else:
                encrypted_doc = sirius_sdk.aries_rfc.EncryptedDocument()
                encrypted_doc.content = content
            await protocol.save_document(structored_doc.id, encrypted_doc)
            output_log_docs.append_display_data(HTML("<b style='color: green;'>Document successfully saved!</b>"))
            
            # Load document
            loaded_structured_document = await protocol.load(structored_doc.id)
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=loaded_structured_document, sequence=0).as_json()
            output_log_docs.append_display_data(HTML("<b style='color: blue;'>Load: Structured document</b>"))
            if loaded_structured_document.doc.encrypted:
                output_log_docs.append_display_data(HTML("<b style='color: red;'>! encrypted !</b>"))
            output_log_docs.append_stdout(json.dumps(jsonable, indent=2))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_docs.append_stderr(repr(e)) 
#################################################################################
#################################################################################
#################################################################################
        

    
#################################################################################
############################ Document removing ###################################
#################################################################################
async def remove_document(doc_id: str):
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # Remove document
            await protocol.remove(uri=doc_id)
            output_log_docs.append_display_data(HTML(f"<b style='color: red;'>Document [{doc_id}] removed</b>"))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_docs.append_stderr(repr(e)) 
#################################################################################
#################################################################################
#################################################################################
        
        
#################################################################################
############################## List documents, filtering with attributes ########
#################################################################################
async def list_documents():
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # List documents
            indexes = await protocol.indexes()
            docs = await indexes.filter(category='document')
            if docs:
                for i, doc in enumerate(docs):
                    output_log_docs.append_display_data(HTML(f"<b style='color: blue;'>Document [{i}]</b>"))
                    jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=doc, sequence=i).as_json()
                    output_log_docs.append_stdout(json.dumps(jsonable, indent=2))
            else:
                output_log_docs.append_display_data(HTML(f"<b style='color: blue;'>Document list is Empty</b>"))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_docs.append_stderr(repr(e)) 
#################################################################################
#################################################################################
#################################################################################

        
def on_btn_createdoc_clicked(b):
    output_log_docs.clear_output()
    doc_name = input_create_doc.value
    doc_content = input_doc_content.value
    asyncio.ensure_future(create_document(doc_name, doc_content.encode('utf-8')))
    
def on_btn_removedoc_clicked(b):
    output_log_docs.clear_output()
    doc_id = input_remove_doc.value
    asyncio.ensure_future(remove_document(doc_id))
    
    
def on_btn_list_clicked(b):
    output_log_docs.clear_output()
    asyncio.ensure_future(list_documents())
    
    
def enc_checkbox_event(ch):
    dropdown_enc_type_doc.layout.display = '' if checkbox_encrypt_doc.value else 'none'
    
    
button_create_doc.on_click(on_btn_createdoc_clicked)
button_remove_doc.on_click(on_btn_removedoc_clicked)
button_list_docs.on_click(on_btn_list_clicked)
checkbox_encrypt_doc.observe(enc_checkbox_event)
display(hbox)

## Upload/Download Streams

In [60]:
import io
import math
import hashlib
import base64


assert VAULT_ID is not None, "You should to select Vault!"
FILE_TO_UPLOAD = None

c = widgets.link
#################################################################################
#################### UI CONTROLS ################################################
#################################################################################
button_create_stream = widgets.Button(description="> Create Stream", button_style='success')
button_upload_file = widgets.FileUpload(multiple=False)
button_list_streams = widgets.Button(description="List Streams >", button_style='primary')
output_log_streams = widgets.Output(layout={"width": "800px", "border": "1px solid #eeeeee", "padding": "10px"})
progress_upload = widgets.FloatProgress(min=0, max=100)
progress_upload.layout.display = 'none'
input_create_stream = widgets.Text(
    value='my_stream.bin',
    placeholder='Type stream name',
    description='Name:  ',
)
checkbox_encrypt_stream = widgets.Checkbox(False, description='Encrypt')
dropdown_enc_type_stream = widgets.Dropdown(
    options=['Wallet', 'SharedKeys'],
    value='Wallet',
    description='EncType:',
    disabled=False,
)
dropdown_enc_type_stream.layout.display = 'none'
input_remove_stream = widgets.Text(
    placeholder='Put here stream id or urn',
    description='ID or URN: '
)
button_remove_stream = widgets.Button(description="> Remove Stream", button_style='danger')
input_load_stream = widgets.Text(
    placeholder='Put here stream id or urn',
    description='ID or URN: '
)
button_load_stream = widgets.Button(description="> Load Stream", button_style='primary')
progress_download = widgets.FloatProgress(min=0, max=100)
output_html = widgets.Output()
progress_download.layout.display = 'none'


vboxes = []

grid_layout = widgets.GridspecLayout(12, 3)
# fill it in with widgets
grid_layout[0, 0] = input_create_stream
grid_layout[0, 1] = button_upload_file
grid_layout[1, 0] = checkbox_encrypt_stream
grid_layout[2, 0] = dropdown_enc_type_stream
grid_layout[1, 1] = button_create_stream
grid_layout[1, 2] = progress_upload

grid_layout[4, 1] = button_list_streams

grid_layout[5, 0] = input_remove_stream
grid_layout[5, 1] = button_remove_stream

grid_layout[7, 0] = input_load_stream
grid_layout[7, 1] = button_load_stream
grid_layout[7, 2] = widgets.HBox([progress_download, output_html])


#grid_layout.layout.width = '500px'


cols = [
    # Each tuple contains a column header and a list of items/widgets
    (
        'Stream Operations', 
        [grid_layout]
        
    ),
    (
        'Stream Logs', 
        [output_log_streams]
    ),
]
for header, data in cols:
    vboxes.append(widgets.VBox([widgets.HTML('<b>%s</b>' % header)] + [
        d if isinstance(d, widgets.Widget) else widgets.HTML(str(d)) for d in data],
    layout=widgets.Layout(border='1px solid #DDDDDD')))
hbox = widgets.HBox(vboxes)

#################################################################################
#################################################################################
#################################################################################


#################################################################################
####################### Create Stream ###########################################
#################################################################################
async def create_stream(name: str, buffer: io.BytesIO, encrypt: bool, content_type: str = None, chunk_size: int = 10*1024):
    buffer.seek(0, io.SEEK_SET)
    content = buffer.read()
    buffer.seek(0, io.SEEK_SET)
    md5 = hashlib.md5(content).hexdigest()
    try:
        async with sirius_sdk.context(CFG):
            jwe: Optional[sirius_sdk.aries_rfc.JWE] = None
            cek: Optional[bytes] = None
            if encrypt:
                if dropdown_enc_type_stream.value == 'Wallet':
                    output_log_streams.append_stdout('\n++++++++++++++++++ Encrypt with Wallet +++++++++++++++++++++')
                    enc = sirius_sdk.aries_rfc.StreamEncryption().setup(target_verkeys=[PROVIDER.me.verkey])
                else:
                    output_log_streams.append_stdout('\n++++++++++++++++++ Encrypt with SharedKeys +++++++++++++++++++++')
                    enc = sirius_sdk.aries_rfc.StreamEncryption().setup(target_verkeys=[PK_SHARED])
                jwe = enc.jwe
                cek = enc.cek
                meta = sirius_sdk.aries_rfc.StreamMeta(
                    content_type=content_type,
                    md5=md5,
                    jwe=jwe.as_json(),
                )
            else:
                meta=sirius_sdk.aries_rfc.StreamMeta(
                    content_type=content_type,
                    md5=md5
                )
            
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            # Create stream
            structured_doc = await protocol.create_stream(
                uri=name,
                meta=meta,
                category='stream'
            )
            
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=structured_doc, sequence=0).as_json()
            output_log_streams.append_display_data(HTML("<b style='color: blue;'>Create: Structured document</b>"))
            output_log_streams.append_stdout(json.dumps(jsonable, indent=2))
            
            # Upload stream
            if jwe:
                stream = await structured_doc.stream.writable(jwe, cek)
            else:
                stream = await structured_doc.stream.writable()
            await stream.open()
            try:
                total_size = buffer.seek(0, io.SEEK_END)
                buffer.seek(0, io.SEEK_SET)
                processed_size = 0
                progress_upload.value = 0.0
                progress_upload.layout.display = ''
                while processed_size < total_size:
                    chunk = buffer.read(chunk_size)
                    await stream.write_chunk(chunk)
                    processed_size += len(chunk)
                    progress_upload.value = 100*processed_size/total_size
                    # REfresh UI
                    await asyncio.sleep(0.1)
                progress_upload.layout.display = 'none'
                output_log_streams.append_display_data(HTML("<b style='color: green;'>Stream successfully saved!</b>"))
            finally:
                await stream.close()

            # Load stream
            loaded_structured_document = await protocol.load(structured_doc.id)
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=loaded_structured_document, sequence=0).as_json()
            output_log_streams.append_display_data(HTML("<b style='color: blue;'>Load: Structured document</b>"))
            output_log_streams.append_stdout(json.dumps(jsonable, indent=2))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_streams.append_stderr(repr(e))
#################################################################################
#################################################################################
#################################################################################


#################################################################################
############################ Stream removing ###################################
#################################################################################
async def remove_stream(doc_id: str):
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # Remove document
            await protocol.remove(uri=doc_id)
            output_log_streams.append_display_data(HTML(f"<b style='color: red;'>Stream [{doc_id}] removed</b>"))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_streams.append_stderr(repr(e)) 
#################################################################################
#################################################################################
#################################################################################


#################################################################################
############################## Load stream ######################################
#################################################################################
async def load_stream(doc_id: str):
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # Load document
            structured_doc = await protocol.load(doc_id)
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=structured_doc, sequence=0).as_json()
            output_log_streams.append_display_data(HTML(f"<b style='color: blue;'>Stream loaded successfully</b>"))
            output_log_streams.append_stdout(json.dumps(jsonable, indent=2))
            
            # Download stream
            progress_download.layout.display = ''
            output_html.clear_output()
            chunks_num = structured_doc.meta.get('chunks', 1)
            #
            jwe_json = structured_doc.meta.get('jwe', None)
            if jwe_json:
                output_log_streams.append_display_data(HTML(f"<b style='color: red;'>Stream was Encrypted</b>"))
                output_log_streams.append_stdout(json.dumps(jwe_json, indent=2))
                jwe = sirius_sdk.aries_rfc.JWE()
                jwe.from_json(jwe_json)
                if dropdown_enc_type_stream.value == 'Wallet':
                    output_log_streams.append_stdout('\n++++++++++++++++++ Decrypt via Wallet +++++++++++++++++++++')
                    stream = await structured_doc.stream.readable(jwe)
                else:
                    output_log_streams.append_stdout('\n++++++++++++++++++ Decrypt via SharedKeys +++++++++++++++++++++')
                    stream = await structured_doc.stream.readable(jwe, keys=sirius_sdk.aries_rfc.KeyPair(PK_SHARED, SK_SHARED))
            else:
                stream = await structured_doc.stream.readable()
            buffer = io.BytesIO()
            counter = 0
            progress_download.value = 0.0
            await stream.open()
            try:
                async for chunk in stream.read_chunked():
                    buffer.write(chunk)
                    counter += 1
                    progress_download.value = 100*counter/chunks_num
                progress_download.layout.display = 'none'
            finally:
                await stream.close()
                
            buffer.seek(0, io.SEEK_SET)
            content = buffer.read()
            buffer.seek(0, io.SEEK_SET)
            md5 = hashlib.md5(content).hexdigest()
            expected_md5 = jwe_json = structured_doc.meta.get('md5', None)
            if expected_md5 == md5:
                output_log_streams.append_display_data(HTML(f"<b style='color: green;'>Stream was successfully DOWNLOADED!!! MD5: {md5}</b>"))
            else:
                output_log_streams.append_display_data(HTML(f"<b style='color: red;'>Stream was DOWNLOADED BUT content mismathed: MD5 actual: {md5} MD5 expected: {expected_md5}</b>"))
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_streams.append_stderr(repr(e)) 
        
#################################################################################
#################################################################################
#################################################################################

#################################################################################
############################## List streams, filtering with attributes ########
#################################################################################
async def list_streams():
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # List structured documents
            indexes = await protocol.indexes()
            docs = await indexes.filter(category='stream')
            if docs:
                for i, doc in enumerate(docs):
                    output_log_streams.append_display_data(HTML(f"<b style='color: blue;'>Stream [{i}]</b>"))
                    jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=doc, sequence=i).as_json()
                    output_log_streams.append_stdout(json.dumps(jsonable, indent=2))
            else:
                output_log_streams.append_display_data(HTML(f"<b style='color: blue;'>Stream list is Empty</b>"))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_streams.append_stderr(repr(e)) 
#################################################################################
#################################################################################
#################################################################################
        

def on_file_changed(inputs):
    global FILE_TO_UPLOAD
    try:
        FILE_TO_UPLOAD = inputs['new']['value'][0]
        input_create_stream.value = FILE_TO_UPLOAD['name']
    except Exception as e:
        if not (isinstance(e, KeyError) or isinstance(e, TypeError)) :
            output_log_streams.append_stderr(repr(e))
            

            
def on_button_create_stream_clicked(b):
    output_log_streams.clear_output()
    if FILE_TO_UPLOAD is None:
        output_log_streams.append_stderr("Select the file to init Stream")
    else:
        name = input_create_stream.value
        content = FILE_TO_UPLOAD['content']
        content_type = FILE_TO_UPLOAD['type']
        buffer = io.BytesIO(content)
        asyncio.ensure_future(
            create_stream(
                name=name,
                buffer=buffer,
                encrypt=checkbox_encrypt_stream.value,
                content_type=content_type
            )
        )

        
def on_button_list_streams(b):
    output_log_streams.clear_output()
    asyncio.ensure_future(list_streams())
    
    
def on_btn_removestream_clicked(b):
    output_log_streams.clear_output()
    doc_id = input_remove_stream.value
    asyncio.ensure_future(remove_stream(doc_id))
    
    
def on_button_load_stream(b):
    output_log_streams.clear_output()
    doc_id = input_load_stream.value
    asyncio.ensure_future(load_stream(doc_id))
    
def on_enc_type_change(ch):
    dropdown_enc_type_stream.layout.display = '' if checkbox_encrypt_stream.value else 'none'

button_upload_file.observe(on_file_changed)
button_create_stream.on_click(on_button_create_stream_clicked)
button_list_streams.on_click(on_button_list_streams)
button_remove_stream.on_click(on_btn_removestream_clicked)
button_load_stream.on_click(on_button_load_stream)
checkbox_encrypt_stream.observe(on_enc_type_change)     

display(hbox)